# Chapter 3: Numerical Computing and Array Programming with NumPy

NumPy is the foundation of numerical computing in Python. In data analytics, you often work with large numeric data (sales amounts, sensor readings, time series, images). NumPy gives you **fast arrays** and **fast math** to analyze that data efficiently.

This chapter is designed for beginners: plain-language explanations, step-by-step code, short exercises, and a small mini-project.

## Introduction

In data analytics, you frequently work with large amounts of numerical data—sales figures, sensor readings, financial time series, or even images (which are just arrays of numbers). Python's built-in lists can handle small datasets, but they become slow and memory-inefficient when dealing with thousands or millions of values.

**NumPy** (Numerical Python) solves this problem by providing:
- **Fast, memory-efficient arrays** called `ndarray`
- **Vectorized operations** that process entire arrays without Python loops
- **Broadcasting** for combining arrays of different shapes
- **Mathematical functions** optimized for numerical computation

NumPy is the foundation upon which many other data science libraries are built, including Pandas, Matplotlib, scikit-learn, and TensorFlow. Understanding NumPy is essential for any data analyst working with Python.

This chapter will guide you through NumPy from the basics to practical applications, with hands-on exercises along the way.

## Learning goals
By the end, you will be able to:
- Explain what an `ndarray` is and why it’s useful
- Create arrays (ranges, filled arrays, random arrays)
- Understand `shape`, `ndim`, and `dtype`
- Index, slice, and mask arrays
- Use vectorized operations and broadcasting
- Apply math/stat functions
- Generate reproducible random numbers
- Perform basic linear algebra
- Make practical performance and memory choices

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print('NumPy:', np.__version__)

# Preferred random number generator (reproducible with a seed)
rng = np.random.default_rng(42)

## 3.1 Introduction to NumPy and `ndarray`

### What is an `ndarray`?
A NumPy array (`ndarray`) is a container for numbers arranged in 1D, 2D, 3D, ... It is optimized for **fast** operations.

### Why not just use Python lists?
Python lists are flexible, but slower for large numeric computations. NumPy arrays store values in a compact block of memory and run math in optimized low-level code.

In [ ]:
lst = [10, 20, 30, 40]
arr = np.array(lst)

print('list:', lst)
print('ndarray:', arr)
print('type:', type(arr))
print('shape:', arr.shape)
print('ndim:', arr.ndim)
print('dtype:', arr.dtype)

## 3.2 Creating arrays
Common creation tools:
- `np.array(...)` from Python data
- `np.arange(start, stop, step)` for integer-like ranges
- `np.linspace(start, stop, num)` for evenly spaced points
- `np.zeros(shape)`, `np.ones(shape)`, `np.full(shape, value)`
- `rng.integers(...)`, `rng.normal(...)` for random arrays

In [ ]:
a = np.arange(0, 10, 2)
b = np.linspace(0, 1, 5)
c = np.zeros((2, 3))
d = np.ones((2, 3), dtype=np.int32)
e = np.full((2, 3), 7)
f = rng.integers(1, 7, size=(3, 4))

print('arange:', a)
print('linspace:', b)
print('zeros:')
print(c)
print('ones int32:')
print(d)
print('full 7:')
print(e)
print('random integers (like dice):')
print(f)

### Exercise (Creating arrays)
1. Create an array of numbers 10..19.
2. Create a 3×3 array filled with 5.
3. Create 11 evenly spaced points from 0 to 1.

Try first, then run the solution cell.

In [ ]:
ex1 = np.arange(10, 20)
ex2 = np.full((3, 3), 5)
ex3 = np.linspace(0, 1, 11)

print('1) ', ex1)
print('2)')
print(ex2)
print('3) ', ex3)

## 3.3 Array attributes and data types
Important attributes:
- `shape`: size per dimension
- `ndim`: number of dimensions
- `size`: total element count
- `dtype`: type of each element (e.g., `int64`, `float32`)
- `nbytes`: memory used by the array data

Why `dtype` matters: it affects **precision**, **memory**, and can cause **overflow** in small integer types.

In [ ]:
x = np.array([[1, 2, 3], [4, 5, 6]], dtype=np.int32)
print(x)
print('shape:', x.shape)
print('ndim:', x.ndim)
print('size:', x.size)
print('dtype:', x.dtype)
print('nbytes:', x.nbytes)

y = x.astype(np.float64)
print()
print('Cast to float64, dtype:', y.dtype, 'nbytes:', y.nbytes)

# Common pitfall: overflow with small integer dtypes
z = np.array([250, 251, 252], dtype=np.uint8)
print()
print('uint8 before:', z)
print('uint8 after +10:', z + 10, '(wraps past 255)')

## 3.4 Indexing, slicing, and masking

- Indexing selects specific positions: `a[0]`, `a[1, 2]`
- Slicing selects ranges: `a[1:5]`, `a[:, 1:3]`
- Masking selects values that match a condition: `a[a > 0]`

Warning: many slices are **views** (they share memory). Fancy indexing often creates a **copy**.

In [ ]:
a = np.arange(1, 13).reshape(3, 4)
print('a:')
print(a)

print()
print('a[0,0]=', a[0, 0])
print('first row:', a[0, :])
print('second column:', a[:, 1])

sub = a[:, 1:3]  # usually a view
print()
print('sub:')
print(sub)
print('sub shares memory (base is not None):', sub.base is not None)

sub[0, 0] = 999
print()
print('after modifying sub, a becomes:')
print(a)

mask = a > 10
print()
print('mask a>10:')
print(mask)
print('values >10:', a[mask])

b = np.arange(10)
picked = b[[1, 3, 5]]  # fancy indexing -> copy
picked[0] = -100
print()
print('b:', b)
print('picked (changed):', picked)

### Exercise (Indexing & masking)
Given `m = np.arange(1, 13).reshape(3, 4)`:
1. Select the last column.
2. Select rows 1..2 and columns 0..1.
3. Replace values < 5 with 0 (without modifying the original).

In [ ]:
m = np.arange(1, 13).reshape(3, 4)
last_col = m[:, -1]
sub = m[1:3, 0:2]

m2 = m.copy()
m2[m2 < 5] = 0

print('m:')
print(m)
print()
print('last column:', last_col)
print('sub:')
print(sub)
print('m after copy+replace (original unchanged):')
print(m2)

## 3.5 Vectorized operations (and 3.10 performance benefits)
Vectorization means you do math on whole arrays at once (no Python loops). This is usually faster and clearer.

Common tools:
- Element-wise arithmetic: `a + b`, `a * 2`, `a ** 2`
- ufuncs: `np.sqrt(a)`, `np.log(a)`, `np.sin(a)`
- conditional logic: `np.where(...)`, `np.clip(...)`

In [ ]:
v = np.array([1, 4, 9, 16], dtype=np.float64)
print('sqrt(v):', np.sqrt(v))

w = np.array([10, 20, 30, 40])
print('w clipped to [15,35]:', np.clip(w, 15, 35))
print('where w>15 then 15 else w:', np.where(w > 15, 15, w))

In [ ]:
# Tiny benchmark (directional): loop vs vectorized
import timeit

n = 100_000
py_list = list(range(n))
np_arr = np.arange(n)

def python_loop_square(lst):
    out = [0] * len(lst)
    for i, val in enumerate(lst):
        out[i] = val * val
    return out

t_py = timeit.timeit(lambda: python_loop_square(py_list), number=3)
t_np = timeit.timeit(lambda: np_arr * np_arr, number=200)

print('Python loop time (3 runs):', round(t_py, 4), 'sec')
print('NumPy vectorized time (200 runs):', round(t_np, 4), 'sec')
print('Note: run counts differ to keep total time reasonable.')

## 3.6 Broadcasting rules
Broadcasting lets NumPy combine arrays of different shapes by “stretching” dimensions of size 1.

Practical rule: compare shapes from the right. A dimension is compatible if it matches or one side is 1.

In [ ]:
M = np.arange(12).reshape(3, 4)
row_vec = np.array([10, 20, 30, 40])
col_vec = np.array([100, 200, 300]).reshape(3, 1)

print('M + row_vec:')
print(M + row_vec)
print()
print('M + col_vec:')
print(M + col_vec)

# Visual broadcasting example: multiple sine curves
x = np.linspace(0, 2 * np.pi, 400)
offsets = np.array([0.0, 0.5, 1.0, 1.5])
y = np.sin(x[None, :] + offsets[:, None])

plt.figure(figsize=(8, 4))
for i, off in enumerate(offsets):
    plt.plot(x, y[i], label=f'offset={off}')
plt.title('Broadcasting: multiple sine curves')
plt.xlabel('x')
plt.ylabel('sin(x + offset)')
plt.grid(True)
plt.legend()
plt.show()

### Exercise (Broadcasting)
Standardize each column: $Z = (A - mean)/std$.
Hint: use `axis=0` and `keepdims=True`.

In [ ]:
A = rng.normal(loc=10, scale=2, size=(5, 3))
mean = A.mean(axis=0, keepdims=True)
std = A.std(axis=0, keepdims=True)
Z = (A - mean) / std

print('Means ~0:', Z.mean(axis=0))
print('Stds ~1:', Z.std(axis=0))

## 3.7 Mathematical and statistical functions
Use `axis` to control whether you compute over rows or columns:
- `axis=0` works down columns
- `axis=1` works across rows

For missing values (`NaN`), use `np.nanmean`, `np.nansum`, etc.

In [ ]:
data = np.array([[1.0, 2.0, np.nan], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])
print('data:')
print(data)
print()
print('NaN-safe column means:', np.nanmean(data, axis=0))
print('NaN-safe row means:', np.nanmean(data, axis=1))
print('90th percentile (ignoring NaN):', np.nanpercentile(data, 90))

## 3.8 Random number generation
Best practice: use `np.random.default_rng(seed)` for reproducible results.
We’ll generate random integers and a normal distribution, then visualize it.

In [ ]:
rng_demo = np.random.default_rng(123)
dice = rng_demo.integers(1, 7, size=20)
normal = rng_demo.normal(0, 1, size=10_000)
print('dice:', dice[:10], '...')

plt.figure(figsize=(7, 3))
plt.hist(normal, bins=40, edgecolor='black')
plt.title('Histogram of N(0,1) samples')
plt.grid(True, alpha=0.2)
plt.show()

# Mini-simulation: estimate pi
rng_mc = np.random.default_rng(2026)
m = 200_000
pts = rng_mc.random((m, 2))
inside = (pts[:, 0] ** 2 + pts[:, 1] ** 2) <= 1.0
pi_est = 4 * inside.mean()
print('Estimated pi:', pi_est)

## 3.9 Linear algebra operations
NumPy provides linear algebra tools in `np.linalg`.

Two common tasks:
- Solve a system $Ax=b$ with `np.linalg.solve`
- Fit a line with least squares `np.linalg.lstsq`

In [ ]:
# Solve Ax = b
A = np.array([[3.0, 1.0], [1.0, 2.0]])
b = np.array([9.0, 8.0])
x = np.linalg.solve(A, b)
print('x:', x)
print('A @ x:', A @ x)

# Least squares line fit y ≈ slope*x + intercept
x_data = np.linspace(0, 10, 30)
y_data = 2.5 * x_data + 1.0 + rng.normal(0, 1.0, size=x_data.shape)
X = np.column_stack([x_data, np.ones_like(x_data)])
coef, residuals, rank, s = np.linalg.lstsq(X, y_data, rcond=None)
slope, intercept = coef
print()
print('slope:', slope, 'intercept:', intercept)

plt.figure(figsize=(7, 3))
plt.scatter(x_data, y_data, s=20, label='data')
plt.plot(x_data, slope * x_data + intercept, color='red', label='fit')
plt.title('Least squares fit')
plt.grid(True, alpha=0.2)
plt.legend()
plt.show()

## 3.11 Memory management and optimization
Practical tips:
- Use smaller dtypes when appropriate (e.g., `float32` vs `float64`)
- Prefer slices (views) when you don’t need a copy
- Use in-place operations when safe (`a *= 2`)
- Use `out=` to avoid extra temporary arrays

In [ ]:
n = 1_000_000
f64 = np.ones(n, dtype=np.float64)
f32 = np.ones(n, dtype=np.float32)
print('float64 bytes:', f64.nbytes)
print('float32 bytes:', f32.nbytes)

x = np.linspace(0, 10, 1000)
out = np.empty_like(x)
np.sin(x, out=out)
print('sin(x) computed with out=, first 5:', out[:5])

## Mini-project: Simple image-like processing with NumPy
We will create a small synthetic 2D “image” (a bright circle), add noise, and apply a simple 3×3 blur using slicing and vectorized operations.

In [ ]:
# Create a synthetic "image" (a bright circle on dark background)
h, w = 120, 160
yy, xx = np.mgrid[0:h, 0:w]
cy, cx = h // 2, w // 2
radius = 35

img = np.zeros((h, w), dtype=np.float32)
circle = (yy - cy) ** 2 + (xx - cx) ** 2 <= radius ** 2
img[circle] = 1.0

# Add Gaussian noise
noise = rng.normal(0, 0.25, size=(h, w)).astype(np.float32)
noisy = np.clip(img + noise, 0.0, 1.0)

# Define a 3x3 Gaussian-like blur kernel
kernel = np.array([[1, 2, 1], 
                   [2, 4, 2], 
                   [1, 2, 1]], dtype=np.float32)
kernel /= kernel.sum()  # Normalize so weights sum to 1

# Apply blur using slicing and vectorized operations
# This is a simple convolution without the edges
blur = noisy.copy()
blur[1:-1, 1:-1] = (
    kernel[0, 0] * noisy[:-2, :-2]   +  # top-left
    kernel[0, 1] * noisy[:-2, 1:-1]  +  # top-center
    kernel[0, 2] * noisy[:-2, 2:]    +  # top-right
    kernel[1, 0] * noisy[1:-1, :-2]  +  # middle-left
    kernel[1, 1] * noisy[1:-1, 1:-1] +  # center
    kernel[1, 2] * noisy[1:-1, 2:]   +  # middle-right
    kernel[2, 0] * noisy[2:, :-2]    +  # bottom-left
    kernel[2, 1] * noisy[2:, 1:-1]   +  # bottom-center
    kernel[2, 2] * noisy[2:, 2:]        # bottom-right
)

# Visualize the results
fig, axes = plt.subplots(1, 3, figsize=(10, 3))
titles = ['Original', 'Noisy', 'Blurred']
images = [img, noisy, blur]

for ax, im, title in zip(axes, images, titles):
    ax.imshow(im, cmap='gray', vmin=0, vmax=1)
    ax.set_title(title)
    ax.axis('off')

plt.tight_layout()
plt.show()

print(f"Image shape: {img.shape}")
print(f"Kernel sum (should be 1.0): {kernel.sum():.4f}")
print(f"Noise range: [{noise.min():.2f}, {noise.max():.2f}]")

## Tips, Warnings, and Common Mistakes

### ✅ Best Practices
1. **Always use vectorized operations** instead of Python loops when possible—they're faster and cleaner.
2. **Choose appropriate dtypes** to save memory (e.g., `float32` instead of `float64` for large datasets).
3. **Use `np.random.default_rng(seed)`** for reproducible random numbers instead of the legacy `np.random.seed()`.
4. **Document your array shapes** in comments, especially when broadcasting is involved.
5. **Use `.copy()` explicitly** when you need an independent copy of data.

### ⚠️ Common Mistakes to Avoid

| Mistake | Problem | Solution |
|---------|---------|----------|
| Modifying a slice without `.copy()` | Changes the original array | Use `arr.copy()` when you need independence |
| Ignoring `dtype` | Integer overflow, precision loss | Check and set `dtype` explicitly |
| Using `==` for float comparison | Floating-point errors | Use `np.isclose()` or `np.allclose()` |
| Forgetting `axis` parameter | Wrong aggregation dimension | Always specify `axis` explicitly |
| Mixing up `shape` dimensions | Wrong results with 2D/3D arrays | Print `shape` frequently during debugging |

### 🔧 Debugging Tips
- **Print shapes often**: `print(arr.shape)` helps catch dimension mismatches early.
- **Check for NaN values**: `np.isnan(arr).any()` reveals hidden missing values.
- **Verify dtype**: Unexpected behavior often comes from integer vs. float dtype differences.
- **Test with small arrays first**: Debug logic on 2×3 arrays before scaling to 1000×1000.

## Summary / Key Takeaways
- NumPy’s `ndarray` enables fast numerical computing.
- Always pay attention to `shape` and `dtype`.
- Prefer vectorized operations and broadcasting over Python loops.
- Use masking to filter and edit data.
- Use `np.nan*` functions when working with missing values.
- Use `default_rng` for reproducible randomness.
- Memory choices (dtype, copies vs views) affect performance.

## Optional references
- NumPy Quickstart: https://numpy.org/doc/stable/user/quickstart.html
- Absolute beginners: https://numpy.org/doc/stable/user/absolute_beginners.html
- Broadcasting: https://numpy.org/doc/stable/user/basics.broadcasting.html
- Random generator: https://numpy.org/doc/stable/reference/random/generator.html